### Build Architecture2 - 모델 디버그를 위한 shape출력
- 레이어가 많아질 경우 어떻게 할 것인가?
- 모든 레이어를 다 작성할 것인지?
- 디버그를 위한 shape 출력문은 어떻게 만들 것인지?
- 여러개의 레이어를 통과시킨 후 merge할 경우

In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd

In [2]:
# 임의의 데이터 생성

input_data = torch.randn((1,3,244,244))

print(input_data.shape)
print(input_data.dtype)

torch.Size([1, 3, 244, 244])
torch.float32


### 리스트 활용
- 리스트(or nn.ModuleList)에 layer를 담아 활용하기
- forward 함수(디버그)
    - for문을 통해 리스트에서 레이어를 하나씩 가져오고 -> 각 레이어마다 shape을 확인

- forward 함수(디버그 완료 후)
    - nn.Sequential

In [3]:
# ModuleList를 활용

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Conv2d(3,5,kernel_size=3),
                  nn.ReLU()]
        self.layers2 = [nn.Conv2d(3,7,kernel_size=3),
                   nn.Conv2d(7,5,kernel_size=1),
                   nn.ReLU()]

        # 두 개의 레이어에 각각 통과시킨 후 merge하기
        self.modules1 = nn.ModuleList(self.layers)
        self.modules2 = nn.ModuleList(self.layers2)

        self.seq1 = nn.Sequential(*self.layers)
        self.seq2 = nn.Sequential(*self.layers2)

        self.fc1 = nn.Linear(5*242*242, 512)

    
    def forward(self, x):
        for idx, module1 in enumerate(self.modules1):
            if idx == 0:
                x1 =  module1(x)
            else:
                x1 = module1(x1)
            print('x1.shape :', x1.shape)

        for idx, module2 in enumerate(self.modules2):
            if idx == 0:
                x2 = module2(x)
            else:
                x2 = module2(x2)
            print('x2 :', x2.shape)

        # merge(addtion)
        conv_out = x1 + x2
        print('conv_out :', conv_out.shape)

        fc_input = conv_out.view(conv_out.size(0), -1)
        print('fc_input:', fc_input.shape)
        fc_output = self.fc1(fc_input)
        print(fc_output.shape)

        return fc_output

In [4]:
model = CNN()
model

CNN(
  (modules1): ModuleList(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (modules2): ModuleList(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (seq1): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (seq2): Sequential(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (fc1): Linear(in_features=292820, out_features=512, bias=True)
)

In [5]:
result = model(input_data)
result

x1.shape : torch.Size([1, 5, 242, 242])
x1.shape : torch.Size([1, 5, 242, 242])
x2 : torch.Size([1, 7, 242, 242])
x2 : torch.Size([1, 5, 242, 242])
x2 : torch.Size([1, 5, 242, 242])
conv_out : torch.Size([1, 5, 242, 242])
fc_input: torch.Size([1, 292820])
torch.Size([1, 512])


tensor([[ 1.1551e-01,  2.5978e-01,  2.3493e-01, -1.8462e-01, -1.6629e-01,
         -2.3492e-01, -2.3347e-01, -1.4957e-01, -1.3519e-01, -4.5818e-02,
          1.0914e-02, -2.0891e-01,  1.0223e-01, -4.3385e-01,  6.3775e-01,
          1.9781e-01,  8.7146e-02, -3.5904e-01, -3.4130e-01,  2.9791e-01,
          1.5553e-01, -2.6370e-01,  3.1170e-01, -9.0019e-02,  5.0368e-02,
          3.0533e-01,  3.0194e-01,  2.3758e-01, -2.0877e-01, -4.6861e-01,
          3.6627e-01,  2.5046e-01,  3.8613e-01, -2.9908e-02,  1.8210e-01,
         -2.0361e-01, -2.9469e-01, -5.3945e-01, -1.2438e-02,  2.8009e-01,
         -2.2920e-01, -1.3573e-01,  5.9221e-01, -3.1375e-01, -3.5598e-01,
         -1.1571e+00,  1.9777e-01,  3.5915e-02,  5.4564e-01, -2.4963e-01,
         -1.0737e-01,  3.0913e-01, -3.4046e-01,  2.8810e-01, -2.1276e-01,
         -1.5422e-01,  2.0147e-01, -1.8585e-01,  7.9911e-02,  2.4307e-01,
          4.0571e-01,  5.5964e-02,  1.8388e-01, -5.2872e-01, -3.1467e-01,
          2.3075e-01, -2.5623e-01,  1.

### Cat n Dog 데이터셋 실습

In [27]:
from pandas.core.frame import DataFrame
paths = []
dataset_type = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/MyDrive/고모부_머신러닝/dogncat'):
    for filename in filenames:
        filepath = dirname + '/' + filename
        paths.append(filepath)

        if '/training_set' in filepath:
            dataset_type.append('train')

        elif '/test_set' in filepath:
            dataset_type.append('test')

        if 'dogs' in filepath:
            labels.append('DOG')
        
        elif 'cats' in filepath:
            labels.append('CAT')

print(len(paths))
print(len(dataset_type))
print(len(labels))

10032
10032
10032


In [28]:
cnd_df = pd.DataFrame({'path' : paths, 'type' : dataset_type, 'label' : labels})
cnd_df.head(20)

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
5,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
6,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
7,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
8,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
9,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [31]:
# '.jpg'를 포함하는 path만 df로 만들기
filter = cnd_df['path'].str.contains('.jpg') 
cnd_df = cnd_df[filter]

In [33]:
cnd_df.shape

(10028, 3)

In [35]:
# train, testset으로 분리
train_df = cnd_df[cnd_df['type'] == 'train']
test_df = cnd_df[cnd_df['type'] == 'test']

train_path = train_df['path'].values
test_path = test_df['path'].values

train_label = train_df['label'].values
test_label = test_df['label'].values

In [38]:
type(cv2.imread(train_path[0]))

numpy.ndarray

In [43]:
# 데이터셋 만들기

from torch.utils.data import Dataset,DataLoader
import cv2

class CndDataset(Dataset):
    def __init__(self, paths, labels):
        super(CndDataset, self).__init__()
        self.path = paths
        self.label = labels

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.path[idx]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244, 244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)

        if self.label is not None:
            label = self.label[idx]

        return image, label

        

In [44]:
cnd_dataset = CndDataset(train_path, train_label)
loader = DataLoader(cnd_dataset, batch_size=10, shuffle=True)
i, l = next(iter(loader))
print(i.view(i.size(0),-1))

tensor([[217., 217., 218.,  ..., 249., 248., 247.],
        [ 14.,  17.,  18.,  ...,  35.,  31.,  32.],
        [125., 125., 124.,  ..., 216., 217., 220.],
        ...,
        [ 55.,  55.,  55.,  ..., 142., 144., 155.],
        [242., 244., 250.,  ...,  83.,  55.,  77.],
        [ 85.,  46.,  31.,  ..., 117.,  98., 133.]])


In [57]:
class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = [nn.Conv2d(3,5,kernel_size=3),
                  nn.ReLU()]
        self.layers2 = [nn.Conv2d(3,7,kernel_size=3),
                   nn.Conv2d(7,5,kernel_size=1),
                   nn.ReLU()]

        # 두 개의 레이어에 각각 통과시킨 후 merge하기
        self.modules1 = nn.ModuleList(self.layers)
        self.modules2 = nn.ModuleList(self.layers2)

        self.seq1 = nn.Sequential(*self.layers)
        self.seq2 = nn.Sequential(*self.layers2)

        self.fc1 = nn.Linear(5 * 242 * 242, 512)

    
    def forward(self, x):
        print('input_image_shape :', x.shape)
        print('input_image_max :', torch.amax(x, dim=(1,2,3)))
        print('input_image_min :', torch.amin(x, dim=(1,2,3)))
        print('input_image_l2_norm :', torch.linalg.vector_norm(x, dim=(1,2,3)))
        print()

        for idx, module1 in enumerate(self.modules1):
            if idx == 0:
                x1 =  module1(x)
            else:
                x1 = module1(x1)
            print('x1.shape :', x1.shape)
            print('x1_max :', torch.amax(x1, dim=(1,2,3)))
            print('x1_min :', torch.amin(x1, dim=(1,2,3)))
            print('x1_l2_norm :', torch.linalg.vector_norm(x1, dim=(1,2,3)))
            print()

        for idx, module2 in enumerate(self.modules2):
            if idx == 0:
                x2 = module2(x)
            else:
                x2 = module2(x2)
            print('x2.shape :', x2.shape)
            print('x2_max :', torch.amax(x2, dim=(1,2,3)))
            print('x2_min :', torch.amin(x2, dim=(1,2,3)))
            print('x2_l2_norm :', torch.linalg.vector_norm(x2, dim=(1,2,3)))
            print()

        # merge(addtion)
        conv_out = x1 + x2
        print('conv_out :', conv_out.shape)
        print('conv_out_max :', torch.amax(conv_out, dim=(1,2,3)))
        print('conv_out_min :', torch.amin(conv_out, dim=(1,2,3)))
        print('conv_out_l2_norm :', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))
        print()
        
        fc_input = conv_out.view(conv_out.size(0), -1)
        print('fc_input:', fc_input.shape)
        print('fc_input_max :', torch.amax(fc_input, dim=(1)))
        print('fc_input_min :', torch.amin(fc_input, dim=(1)))
        print('fc_input_l2_norm :', torch.linalg.vector_norm(fc_input, dim=(1)))
        print()

        fc_output = self.fc1(fc_input)
        print('fc_output :', fc_output.shape)
        print('fc_output_max :', torch.amax(fc_output, dim=(1)))
        print('fc_output_min :', torch.amin(fc_output, dim=(1)))
        print('fc_output_l2_norm :', torch.linalg.vector_norm(fc_output, dim=(1)))
        print()

        # return fc_output

In [58]:
cnd_model = CNN2()
cnd_model

CNN2(
  (modules1): ModuleList(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (modules2): ModuleList(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (seq1): Sequential(
    (0): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (seq2): Sequential(
    (0): Conv2d(3, 7, kernel_size=(3, 3), stride=(1, 1))
    (1): Conv2d(7, 5, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU()
  )
  (fc1): Linear(in_features=292820, out_features=512, bias=True)
)

### feature map 데이터의 특성을 파악하기

- 각 배치 내에서 모든 과정의 min 값과 max값을 출력

- 각 배치 내에서 모든 과정의 l2 norm을 프린트해보기

In [59]:
image_array, image_label = next(iter(loader))
cnd_model(image_array)

input_image_shape : torch.Size([10, 3, 244, 244])
input_image_max : tensor([255., 255., 255., 242., 249., 253., 255., 255., 255., 255.])
input_image_min : tensor([0., 4., 1., 0., 0., 0., 0., 0., 0., 0.])
input_image_l2_norm : tensor([54412.6055, 67641.2422, 52337.1367, 50719.5391, 59310.3633, 45167.4453,
        53339.7852, 72071.0703, 43673.6094, 62296.3203])

x1.shape : torch.Size([10, 5, 242, 242])
x1_max : tensor([218.3160, 252.0642, 228.9841, 203.2696, 196.6485, 219.1621, 277.1967,
        222.3608, 233.2589, 239.1934], grad_fn=<AmaxBackward0>)
x1_min : tensor([-129.0866, -164.9603, -137.0276, -118.7575, -105.3153, -139.9174,
        -220.2619, -138.9643, -149.8814, -146.0530], grad_fn=<AminBackward0>)
x1_l2_norm : tensor([30677.2031, 38855.7578, 29369.9277, 27775.2031, 34202.7656, 26326.6367,
        31269.0137, 41243.7266, 25910.0488, 35890.1680],
       grad_fn=<LinalgVectorNormBackward0>)

x1.shape : torch.Size([10, 5, 242, 242])
x1_max : tensor([218.3160, 252.0642, 228.9841, 


### Activation Neuron 확인

- 1. 몇개의 뉴런이 activation되는지 print 
- 2. activation map을 확인하는 과정이 중요
    - relu를 지나고 나서 어떤 neuron이 activation이 되는지 확인하는 과정

- 3. class activation map
    - 클래스가 들어왔을 떄 어떤 뉴런이 activation 되는지
    - ex) 개일떄 활성화되는 뉴런과 고양이 일 때 활성화되는 뉴런이 다를 때 가장 좋은 것


In [38]:
layers = [nn.Conv2d(3, 10, [5,5], padding='same')]
layers.append(nn.ReLU())
layers.append(nn.Conv2d(10, 20, [5,5], stride=2))
layers.append(nn.ReLU())


In [39]:
modules = nn.ModuleList(layers)

In [40]:
seq = nn.Sequential(*layers)

In [41]:
input = torch.randn([1,3,224,224])
print(input.shape)
print(input.dtype)
print(type(input))

torch.Size([1, 3, 224, 224])
torch.float32
<class 'torch.Tensor'>


In [42]:
out = seq(input)

In [43]:
out.shape

torch.Size([1, 20, 110, 110])

In [44]:
o = modules[0](input)
o.shape

torch.Size([1, 10, 224, 224])

In [49]:
o = modules[1](input)
o.shape

torch.Size([1, 3, 224, 224])

In [50]:
o = modules[2](0)

TypeError: ignored

In [ ]:
# 디버그하는 방법. list를 돌면서 하나씩 확인

def forward(input):
    x = input
    for module in modules:
        x = module(x)
        print()

In [ ]:
# 디버그가 완료된 후 nn.Sequential을 사용

def forward(input):
    x = input
    for layer in seq:
        x = module(x)
    return x

### ResNet 구현해보기
- 참조  
    https://towardsdev.com/implement-resnet-with-pytorch-a9fb40a77448

In [93]:
class PrintShape(nn.Module):
    def __init__(self):
        super(PrintShape, self).__init__()

    def forward(self, x):
        print(x.shape)
        return x

In [121]:
from torch.nn.modules.batchnorm import BatchNorm2d
import torch.nn.functional as F
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample):
        super(ResBlock, self).__init__()
        if downsample:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=2),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
            self.shortcut = nn.Sequential(
                # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1),
                # nn.BatchNorm2d(out_channels)
            )

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        shortcut = self.shortcut(x)
        # print('shortcut :', shortcut.size())
        x = self.bn1(self.conv1(x))
        x = F.ReLU(x)
        # print('conv1 :', x.size())
        x = self.bn2(self.conv2(x))
        x = F.ReLU(x)
        # print('conv2 :', x.size())
        output = shortcut + x
        # print('output :', x.size())
        return output


In [122]:
class ResNet18(nn.Module):
    def __init__(self, in_channels, resblock, outputs=1000):
        super(ResNet18, self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3), # 7x7 kernel을 사용하고 output shape을 1/2로 줄이기 위해서는 padding이 필요
            nn.MaxPool2d(3, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )

        self.layer1 = nn.Sequential(
            resblock(64, 64, downsample=False),
            resblock(64, 64, downsample=False)
        )

        self.layer2 = nn.Sequential(
            resblock(64, 128, downsample=True),
            resblock(128, 128, downsample=False)
        )

        self.layer3 = nn.Sequential(
            resblock(128, 256, downsample=True),
            resblock(256, 256, downsample=False)
        )
        
        self.layer4 = nn.Sequential(
            resblock(256, 512, downsample=True),
            resblock(512, 512, downsample=False)
        )

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear(512, 10)
        
    def forward(self, x):
        x = self.layer0(x),
        # print(x.size())
        x = self.layer1(x),
        x = self.layer2(x),
        x = self.layer3(x),
        x = self.layer4(x),
        x = self.gap(x),
        output = self.fc(x)
        # print('output :', output.size())

        return x

In [123]:
from torchsummary import summary

resnet18 = ResNet18(3, ResBlock, outputs=1000)
summary(resnet18, input_data)

TypeError: ignored

### 후기

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:  
이거 왜 자꾸 나오는거냐..